<a href="https://colab.research.google.com/github/MK316/workspace/blob/main/ASR01/ASR04_RecRate_SK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recognition Rate calculation using nltk

Need a recognition result file and rainbow sentence file

* **SE.zip** (gTTS generated audio files) - completed (asof 0106), [SETR_result.csv](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SETR_result.csv)
* **SK.zip** (gTTS generated audio files) - completed (asof 0106), [SKTR_result.csv](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SKTR_result.csv)
* **SK.zip 2** with medium.en model [SKTR_result_m.csv](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SK_result_m.csv)
* HE.zip (Native production)
* HK.zip (Learners' production)

## Perception rate ~ Error rate

* Perception rate: Percentage of words that match the original text

### Demo

- Sample text: "**The rainbow** is a division of white light into many beautiful colors." (12 words)

- Perceived text: **The lane bowl** is a division of white light into many beautiful colors. (11 words)

Perception rate = (11/12)*100 = 91%  
Error rate = 9%

### Read the rainbow text (19 sentences)

[The rainbow passage sentence file list from github](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentences.csv)

SR result file from github: [SK](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SK_result_0106.csv), [SE](https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SE_result_0106.csv)

In [7]:
import pandas as pd

url = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/data/rainbow_sentences.csv"
rb = pd.read_csv(url)
print(rb.head())

url2 = "https://raw.githubusercontent.com/MK316/workspace/main/ASR01/results/SK_result_m.csv"
sk = pd.read_csv(url2)
print(sk.head())



   SN                                           Sentence
0  S1  When the sunlight strikes raindrops in the air...
1  S2  The rainbow is a division of white light into ...
2  S3  These take the shape of a long round arch, wit...
3  S4  There is, according to legend, a boiling pot o...
4  S5             People look, but no one ever finds it.
   Filename  Runtime                                         Recognized
0  SK01.wav    5.625   When the sunlight strikes raindrops in the ai...
1  SK02.wav    1.035   Ta rainbow EJ division of white light into ma...
2  SK03.wav    5.222   T-z take the shape away long round arch, with...
3  SK04.wav    1.112   Theor is, according to legend, a building put...
4  SK05.wav    4.720                                making new personal


In [8]:
# Add original text to the result file

oritext = list(rb['Sentence'])
sn = list(rb['SN'])

sk['Sentence'] = oritext
sk['SN'] = sn
sk.head()

,Filename,Runtime,Recognized,Sentence,SN
0,SK01.wav,5.625,When the sunlight strikes raindrops in the ai...,When the sunlight strikes raindrops in the air...,S1
1,SK02.wav,1.035,Ta rainbow EJ division of white light into ma...,The rainbow is a division of white light into ...,S2
2,SK03.wav,5.222,"T-z take the shape away long round arch, with...","These take the shape of a long round arch, wit...",S3
3,SK04.wav,1.112,"Theor is, according to legend, a building put...","There is, according to legend, a boiling pot o...",S4
4,SK05.wav,4.720,making new personal,"People look, but no one ever finds it.",S5


Column ordering & save it as csv (e.g., 'SER_result.csv')

In [9]:
sk = sk[['SN', 'Filename', 'Runtime', 'Sentence', 'Recognized']]
cols = list(sk.columns.values)
sk.to_csv('SKR_result.csv', index=False)
sk.head()

,SN,Filename,Runtime,Sentence,Recognized
0,S1,SK01.wav,5.625,When the sunlight strikes raindrops in the air...,When the sunlight strikes raindrops in the ai...
1,S2,SK02.wav,1.035,The rainbow is a division of white light into ...,Ta rainbow EJ division of white light into ma...
2,S3,SK03.wav,5.222,"These take the shape of a long round arch, wit...","T-z take the shape away long round arch, with..."
3,S4,SK04.wav,1.112,"There is, according to legend, a boiling pot o...","Theor is, according to legend, a building put..."
4,S5,SK05.wav,4.720,"People look, but no one ever finds it.",making new personal


## Recognition rate and finding mismatching words using NLTK

In [10]:
#@markdown Calculate recognition rate, record missing words => dataframe (SKTR_result.csv) 
from nltk.tokenize import RegexpTokenizer

nw = []
rw = []
rr = []
mw = []


for i in range(0,len(sk['SN'])):
  t1 = sk['Sentence'][i]

# text 1
  txt1 = t1.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist = tokenizer.tokenize(txt1)
  nt = len(wlist)
  nw.append(nt)
# text 2
  t2 = sk['Recognized'][i]
  txt2 = t2.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  wlist1 = tokenizer.tokenize(txt2)
  nt1 = len(wlist1)
  rw.append(nt1)

# Recognition rate

  from tables.idxutils import calc_chunksize
  from nltk.downloader import ErrorMessage
# txt1(original text), txt2 (recognized text)
 
  mword = []
  score = 0
  for i in range(0, len(wlist1)):
      w = wlist1[i]

      if w in wlist:
        sc = 1
      else:
        sc = 0
        mword.append(w)
      mwords = ' '.join(mword)
      score = score + sc
  mw.append(mwords)
  missingword = (score/len(wlist))*100

  # RecRate = float(format(missingword, '.2f'))
  # ErrRate = float(format((100.0 - RecRate), '.2f'))
  
  rr.append(missingword)
  # print('Matching words: %d'%score, 'out of %d words'%len(wlist))
  # print("="*50)
  # print('Recognition Rate: %f %%'%RecRate)
  # print('Error Rate: %f %%'%ErrRate)

sk['NWord'] = nw
sk['RecWord'] = rw
sk['RecRate'] = rr
sk['Missingwords'] = mw

sk.to_csv('SKTR_result.csv')

sk.tail()


,SN,Filename,Runtime,Sentence,Recognized,NWord,RecWord,RecRate,Missingwords
14,S15,SK15.wav,1.041,Many complicated ideas about the rainbow have ...,Many complicated ideas about Dao Renbou have ...,9,9,77.777778,dao renbou
15,S16,SK16.wav,2.334,The difference in the rainbow depends consider...,The difference in the rainbow depends conside...,28,26,82.142857,traps traps traps
16,S17,SK17.wav,1.262,The actual primary rainbow observed is said to...,The actual primary rainbow observed is said t...,19,20,94.736842,inposition posts
17,S18,SK18.wav,2.450,If the red of the second bow falls upon the gr...,If the red over the second ball pulls open th...,36,35,75.000000,over ball pulls open over keep able from
18,S19,SK19.wav,2.114,"This is a very common type of bow, one showing...",This is a very common type of boot. Once you ...,21,22,47.619048,boot once you have made you need to clean up t...


The end